<a href="https://colab.research.google.com/github/rlrocha90/CursoVW/blob/main/Esteira%20Prod/Roteiro_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Roteiro 6
Produção

In [ ]:
!pip install tensorflow-transform

In [ ]:
import tensorflow_transform as tft

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

START_TOKEN_ID = 101
END_TOKEN_ID = 102
TFHUB_URL = ("https://www.kaggle.com/models/tensorflow/bert/tensorFlow2/en-uncased-l-12-h-768-a-12/3")

In [ ]:
def load_bert_model(model_url=TFHUB_URL):
  bert_layer = hub.KerasLayer(handle=model_url, trainable=False)
  return bert_layer

In [ ]:
def _preprocessing_fn(inputs):
  vocab_file_path = load_bert_model().resolved_object.vocab_file.asset_path

  bert_tokenizer = tf_text.BertTokenizer(
    vocab_lookup_table=vocab_file_path,
    token_out_type=tf.int64,
    lower_case=True)

  text = inputs['message']
  category = inputs['category']

  # Normalize text
  text = tf_text.normalize_utf8(text)

  # Tokenization
  tokens = bert_tokenizer.tokenize(text).merge_dims(1, -1)

  # Add control tokens
  tokens, input_type_ids = tf_text.combine_segments(tokens, start_of_sequence_id=START_TOKEN_ID, end_of_segment_id=END_TOKEN_ID)

  # Token truncation / padding
  tokens, input_mask_ids = tf_text.pad_model_inputs(tokens, max_seq_length=128)

  # Convert categories to labels
  labels = tft.compute_and_apply_vocabulary(label, vocab_filename="category")

  return {
    "labels": labels,
    "input_ids": tokens,
    "input_mask_ids": input_mask_ids,
    "input_type_ids": input_type_ids,
  }

In [ ]:
!pip install -U "jax[cpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html -q
!pip install --upgrade keras -q

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras
from keras.datasets import reuters
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras import layers
num_words = 1000
print(f'Keras version: {keras.__version__}\n\n')
(reuters_train_x, reuters_train_y), (reuters_test_x, reuters_test_y) = reuters.load_data(num_words=num_words)
n_labels = np.unique(reuters_train_y).shape[0]
reuters_train_y = to_categorical(reuters_train_y, 46)
reuters_test_y = to_categorical(reuters_test_y, 46)

In [ ]:
reuters_train_x = sequence.pad_sequences(reuters_train_x, maxlen=20)
reuters_test_x = sequence.pad_sequences(reuters_test_x, maxlen=20)

In [ ]:
model1 = keras.Sequential(
    [
        layers.Embedding(num_words, 1000),
        layers.Flatten(),
        layers.Dense(256),
        layers.Dropout(0.25),
        layers.Activation('relu'),
        layers.Dense(46),
        layers.Activation('softmax')
    ]
)

In [ ]:
model1.compile(loss="categorical_crossentropy", optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs_model1")
model_1 = model1.fit(reuters_train_x, reuters_train_y,validation_data=(reuters_test_x, reuters_test_y),batch_size=128, epochs=20, verbose=1,callbacks=[tensorboard_callback])

In [ ]:
#Load the TensorBoard notebook extension
%load_ext tensorboard
# Open an embedded TensorBoard viewer
%tensorboard --logdir ./logs_model1

In [ ]:
model2 = keras.Sequential(
  [
    layers.Embedding(num_words, 10),
    layers.Flatten(),
    layers.Dense(256),
    layers.Dropout(0.25),
    layers.Activation('relu'),
    layers.Dense(46),
    layers.Activation('softmax')
  ]
)

In [ ]:
model2.compile(loss="categorical_crossentropy", optimizer="rmsprop",metrics=["accuracy"])
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs_model2")
model_2 = model2.fit(reuters_train_x, reuters_train_y, validation_data=(reuters_test_x, reuters_test_y), batch_size=128, epochs=20, verbose=1, callbacks=[tensorboard_callback])

In [ ]:
# Open an embedded TensorBoard viewer
%tensorboard --logdir ./logs_model2